# 包库引用

In [2]:
import csv, sys, os, math
import pandas as pd
import numpy as np
from numpy import argmax
import random
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score,roc_curve,auc  
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import pyplot
from functools import reduce
from pathlib import Path

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import DMatrix

# 数据拼接

In [3]:
workdir = os.path.abspath('.')
#cli_data = pd.read_csv('%s/BRCA_clinical.tsv'%workdir,sep='\t')
exp_data = pd.read_csv('%s/BRCA.sampleMap_HiSeqV2'%workdir,sep='\t')
gene_name = exp_data['sample']
#gene_name.to_csv('%s/gene_name.txt'%workdir,sep='\t') #写入文件保存
exp_data = exp_data.drop(labels = 'sample',axis=1).transpose()
exp_data.columns = gene_name

ylable= pd.DataFrame(pd.DataFrame(exp_data.index)[0].str.split('-').str[-1])
exp_data['ylable'] = np.where(ylable[0].str.contains("0"), 1, 0)
exp_data.to_csv('%s/BRCA.txt'%workdir,sep='\t',index='sample')

x = exp_data.iloc[:,0:20531]
x = x.drop(columns = 'ylable')
y = exp_data.iloc[:,-1].astype(int)

# ModelBuilding

In [4]:
%%time
importance_list = []
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.2,random_state = 32, stratify=y)
params = {
        'booster' : 'gbtree',
        'objective' : 'binary:logistic',
        #eval_metric : 'auc',  
        #'max_depth' : 5,  
        #lambda = 10,
        'subsample' : 0.5,
        'colsample_bytree' : 0.75,
        #'min_child_weight' : 3,
        #'eta' : 0.025,
        'seed' : 0,
        #'nthread' : 8,
        #'gamma' : 0.15,
        'learning_rate' : 0.005}
model = xgb.XGBClassifier(**params)
model.fit(x_train,y_train)


CPU times: total: 5min 1s
Wall time: 41.9 s


In [29]:
combine = zip(x.columns,model.feature_importances_)
importance = {k: v for k, v in combine}
importance_list = []
importance_list.append(importance)
df_importance = pd.DataFrame(importance_list).fillna(0)
df_result = df_importance.mean(axis =0).sort_values(ascending= 0)

In [35]:
df_result

MME         0.050613
TMEM220     0.047174
PPP1R12B    0.047012
ADAMTS5     0.045806
SDPR        0.045141
              ...   
ADAMTSL1    0.000000
PTGER2      0.000000
ADAMTSL3    0.000000
ADAMTSL2    0.000000
SELS        0.000000
Length: 20530, dtype: float32

# Checking

In [36]:
#十次cv分别每次实验的特征重要性数值（无序）
df_importance

ARHGEF10L     HIF3A  RNF17  RNF10     RNF11  RNF13  GTF2IP1  REM1  MTVR2  \
0        0.0  0.004344    0.0    0.0  0.000042    0.0      0.0   0.0    0.0   

   RTN4RL2  ...  TULP2  NPY5R  GNGT2  GNGT1  TULP3  PTRF  BCL6B  GSTK1  SELP  \
0      0.0  ...    0.0    0.0    0.0    0.0    0.0   0.0    0.0    0.0   0.0   

   SELS  
0   0.0  

[1 rows x 20530 columns]

In [37]:
#最终的特征重要性（有序）
df_result

MME         0.050613
TMEM220     0.047174
PPP1R12B    0.047012
ADAMTS5     0.045806
SDPR        0.045141
              ...   
ADAMTSL1    0.000000
PTGER2      0.000000
ADAMTSL3    0.000000
ADAMTSL2    0.000000
SELS        0.000000
Length: 20530, dtype: float32

In [38]:
list(df_result.index)#有序版

['MME',
 'TMEM220',
 'PPP1R12B',
 'ADAMTS5',
 'SDPR',
 'SPRY2',
 'SCN4B',
 'FIGF',
 'CD300LG',
 'NKAPL',
 'MMP11',
 'ARHGAP20',
 'MAMDC2',
 'LMOD1',
 'COL10A1',
 'MFAP2',
 'ALDH18A1',
 'PAMR1',
 'SEMA3G',
 'INMT',
 'GABARAPL1',
 'TGFBR2',
 'COL11A1',
 'ELMO2',
 'FAM162B',
 'ME3',
 'RRAD',
 'HPN',
 'MATN3',
 'GABRD',
 'TRIM34',
 'C3orf70',
 'PEX11B',
 'SCARA5',
 'GPRASP1',
 'PYCR1',
 'PHKG1',
 'SCHIP1',
 'F2R',
 'MMP13',
 'ZNF502',
 'KLHL12',
 'OLFM1',
 'MR1',
 'PKHD1L1',
 'MGRN1',
 'OVCH2',
 'RASL11B',
 'HIF3A',
 'CAPN11',
 'CALCOCO1',
 'C19orf23',
 'LMBR1L',
 'C19orf21',
 'psiTPTE22',
 'HHIPL2',
 'PLSCR4',
 'KIAA0247',
 'KCTD3',
 'GRIA4',
 'PLXNA3',
 'BPNT1',
 'TMEM171',
 'UNC5B',
 'FAM189A2',
 'GTSE1',
 'C19orf76',
 'SCN2B',
 'OXTR',
 'WISP1',
 'SMYD1',
 'PENK',
 'GFAP',
 'ASF1B',
 'KLF6',
 'IGFBP1',
 'BGN',
 'LRRC3B',
 'KCNE1',
 'ITPRIPL1',
 'GNAL',
 'SLC17A7',
 'IGF2AS',
 'C17orf97',
 'C10orf107',
 'IQGAP3',
 'LRRTM2',
 'SNX27',
 'EIF2B4',
 'KCNS3',
 'FLRT3',
 'FAM126A',
 'NBL1',
 

In [39]:
name=['XGBoost']
name2=range(1,len(df_result.index)+1)
test=pd.DataFrame(columns=name,index=name2,data=df_result.index)
print(test)

        XGBoost
1           MME
2       TMEM220
3      PPP1R12B
4       ADAMTS5
5          SDPR
...         ...
20526  ADAMTSL1
20527    PTGER2
20528  ADAMTSL3
20529  ADAMTSL2
20530      SELS

[20530 rows x 1 columns]


In [40]:
test.to_csv('xgb特征排序结果.csv')